In [ ]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.linear_model import LogisticRegression, Ridge, RidgeClassifier

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
!pip install catboost
try:
    os.chdir("./drive/My Drive/Финал БДиМО 2021/Финал")
except: pass

Mounted at /content/drive
     |████████████████████████████████| 65.7MB 50kB/s 


In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
rheed_long = pd.read_csv("ONTI_Dataset_parsed.csv", index_col="Unnamed: 0")

In [ ]:
rheed_long.head()

,Coolness_RHEED,Length_RHEED,R FWHM_RHEED,Speed_RHEED,Time_RHEED,X FWHM_RHEED,Y FWHM_RHEED,Crystal Position,Displayed Rate,Filtered Rate,Fundamental Freq,Length,Raw Rate,Source Power,Speed,Time
0,0.0,0.003721,0.0,0.0406,2021-02-23 02:21:05,0.0,0.0,1.0,6.027,6.000,inf,0.003721,5.990,21.64,NaN,2021-02-23 02:21:05
1,0.0,0.028870,0.0,0.4172,2021-02-23 02:21:06,0.0,0.0,1.0,6.070,6.094,inf,0.028870,6.110,21.06,NaN,2021-02-23 02:21:06
2,0.0,0.040400,0.0,0.5366,2021-02-23 02:21:07,0.0,0.0,1.0,6.156,6.082,inf,0.040400,6.168,20.97,NaN,2021-02-23 02:21:07
3,0.0,0.067750,0.0,0.7820,2021-02-23 02:21:08,0.0,0.0,1.0,6.110,6.168,inf,0.067750,6.156,20.70,NaN,2021-02-23 02:21:08
4,0.0,0.083560,0.0,0.8794,2021-02-23 02:21:09,0.0,0.0,1.0,6.094,6.168,inf,0.083560,6.145,20.60,NaN,2021-02-23 02:21:09


In [ ]:
#важные колонки для использования
val_col = ['Coolness_RHEED',  'R FWHM_RHEED',  'X FWHM_RHEED', 'Y FWHM_RHEED',
            'Filtered Rate', 'Displayed Rate', 'Raw Rate',
            'Source Power',  "Length", "Crystal Position"]

In [ ]:
#создаем датасет разными аггрегационными функциями
def generated_dataset(series, wind_size=5, min_period=1):
    result = series.rolling(wind_size, min_periods=min_period).agg(["median", "mean", "std", "var"])
    print(result.shape)
    delta = series - series.shift(wind_size)
    print(delta.shape)
    result = result.join(delta)
    result.rename(columns={c:str(c) + "_" + str(wind_size)
        for c in result.columns}, inplace=True)

    print(result.shape)

    return result

In [ ]:
%%time
df_X = generated_dataset(rheed_long[val_col], wind_size=5).astype(np.float16)

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 11.6 s, sys: 512 ms, total: 12.1 s
Wall time: 12.1 s


In [ ]:
%%time
df_X = df_X.join(generated_dataset(rheed_long[val_col], wind_size=50)).astype(np.float16)
df_X.shape

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 15.7 s, sys: 95.4 ms, total: 15.7 s
Wall time: 15.7 s


In [ ]:
%%time
df_X = df_X.join(generated_dataset(rheed_long[val_col], wind_size=150)).astype(np.float16)
df_X.shape

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 16.7 s, sys: 158 ms, total: 16.9 s
Wall time: 16.8 s


In [ ]:
%%time
df_X = df_X.join(generated_dataset(rheed_long[val_col], wind_size=250)).astype(np.float16)
df_X.shape

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 17.2 s, sys: 160 ms, total: 17.4 s
Wall time: 17.3 s


In [ ]:
%%time
df_X = df_X.join(generated_dataset(rheed_long[val_col], wind_size=350)).astype(np.float16)
df_X.shape

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 18.1 s, sys: 188 ms, total: 18.3 s
Wall time: 18.2 s


In [ ]:
%%time
df_X = df_X.join(generated_dataset(rheed_long[val_col], wind_size=500)).astype(np.float16)
df_X.shape

(2363772, 40)
(2363772, 10)
(2363772, 50)
CPU times: user 18.5 s, sys: 215 ms, total: 18.8 s
Wall time: 18.7 s


In [ ]:
mask_base = df_X.isna().sum(axis=1) == 0
df_X.shape

(2363772, 300)

In [ ]:
mask_base.sum()

2288459

In [ ]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X, y,
                         test_size=0.3, val_size=0.3,
                         random_state=42, shuffle=True):
    """
    separator for train, test, validation
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=1 - (test_size + val_size),
                                                        random_state=random_state, shuffle=shuffle)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test,
                                                    test_size=test_size / (test_size + val_size),
                                                    random_state=random_state, shuffle=shuffle)
    return X_train, y_train, X_test, y_test, X_val, y_val

In [ ]:
wind_size = 435
df = df_X[wind_size:-wind_size]

cool_col = "('Coolness_RHEED', 'mean')_50"
y = (np.abs(df[cool_col] - df[cool_col].shift(-wind_size))) > 5

mask = mask_base
df = df[mask]
y = y[mask]

In [ ]:
y.value_counts()

False    2188727
True       99732
Name: ('Coolness_RHEED', 'mean')_50, dtype: int64

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(df,
                                                                      y,
                                                                      test_size=0.2,
                                                                      val_size=0.1,
                                                                      shuffle=True)

In [ ]:
from catboost import CatBoostClassifier

coef = 100
model = CatBoostClassifier(iterations=coef,
                          random_state=42,
                          verbose=1,
                          task_type="GPU",
                          max_depth=6)

In [ ]:
model.fit(X_train,
          y_train)

Learning rate set to 0.176815
0:	learn: 0.4166661	total: 116ms	remaining: 11.4s
1:	learn: 0.2804967	total: 210ms	remaining: 10.3s
2:	learn: 0.2030046	total: 313ms	remaining: 10.1s
3:	learn: 0.1628323	total: 411ms	remaining: 9.86s
4:	learn: 0.1401408	total: 503ms	remaining: 9.56s
5:	learn: 0.1286482	total: 583ms	remaining: 9.14s
6:	learn: 0.1179811	total: 676ms	remaining: 8.98s
7:	learn: 0.1125318	total: 755ms	remaining: 8.68s
8:	learn: 0.1069295	total: 847ms	remaining: 8.56s
9:	learn: 0.1039090	total: 943ms	remaining: 8.49s
10:	learn: 0.1011423	total: 1.03s	remaining: 8.37s
11:	learn: 0.0992598	total: 1.12s	remaining: 8.18s
12:	learn: 0.0979779	total: 1.2s	remaining: 8.03s
13:	learn: 0.0967204	total: 1.27s	remaining: 7.83s
14:	learn: 0.0953533	total: 1.37s	remaining: 7.76s
15:	learn: 0.0941894	total: 1.45s	remaining: 7.63s
16:	learn: 0.0929737	total: 1.55s	remaining: 7.59s
17:	learn: 0.0919592	total: 1.66s	remaining: 7.56s
18:	learn: 0.0906279	total: 1.74s	remaining: 7.41s
19:	learn: 0

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from collections import Counter

In [ ]:
proba_test = model.predict_proba(X_test)[:, 1]

In [ ]:
print(f1_score(y_test, proba_test > 0.5))
print(roc_auc_score(y_test, proba_test))

0.7684204499914123
0.9776358219044708


In [ ]:
Counter(proba_test > 0.4)

Counter({False: 441103, True: 16589})

In [ ]:
proba_val = model.predict_proba(X_val)[:, 1]

In [ ]:
print(f1_score(y_val, proba_val > 0.5))
print(roc_auc_score(y_val, proba_val))

0.7695929264511685
0.9791504881113492


In [ ]:
Counter(proba_val > 0.4)

Counter({False: 220534, True: 8312})

In [ ]:
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)